In [1]:
import urllib
import bs4
import pandas
import numpy as np
from urllib import request
import requests

### Scrapping de la page Yazio 

On commence notre travail de scrapping de la page yazio qui est en open data. 
On utilise pour cela l'outil BeautifulSoup qui rend le contenu html lisible. 
La page que nous scrappons est une interface présentant plusieurs catégories d'aliments (boissons, boissons alcoolisées, fruits...). 
Dès qu'on clique sur une catégorie, on a accès à des sous-catégories. 
Dès qu'on clique sur une sous-catégorie, on a accès à de nombreux aliments. 
Lorsqu'on clique sur ces aliments, on obtient ses macro-nutriments. 
Ce sont ces macro-nutriments que nous souhaitons récolter in fine sur une database. 

In [34]:
yaziopage = requests.get("https://www.yazio.com/fr/aliments")

In [35]:
clean_yazio_page = bs4.BeautifulSoup(yaziopage.content, "lxml")

On observe comment l'écriture du site est réalisé, afin de voir comment récupérer les liens vers lesquels nous sommes redirigés lorsqu'on clique sur une catégorie d'aliment. 

In [40]:
# Récupération des liens pour chaque catégorie

elem = clean_yazio_page.findAll("a",{"class": "position-relative text-decoration-none"})

for element in range(2):
    print(elem[element])

<a class="position-relative text-decoration-none" href="https://www.yazio.com/fr/aliments/boissons-alcoolisees" title="Boissons alcoolisées">
<picture data-alt="Boissons alcoolisées" data-default-src="https://images.yazio.com/drinksalcoholic.jpg?w=156&amp;h=88&amp;cf">
<source srcset="https://images.yazio.com/drinksalcoholic.jpg?w=156&amp;h=88&amp;cf,
                                https://images.yazio.com/drinksalcoholic.jpg?w=312&amp;h=176&amp;cf 2x">
<img alt="Boissons alcoolisées" class="br-12" src="https://images.yazio.com/drinksalcoholic.jpg?w=312&amp;h=176&amp;cf" title="Boissons alcoolisées"/>
</source></picture>
<div class="mt-8 text-left">
<span class="textlink light">Boissons alcoolisées</span>
</div>
</a>
<a class="position-relative text-decoration-none" href="https://www.yazio.com/fr/aliments/boissons-non-alcoolisees" title="Boissons non-alcoolisées">
<picture data-alt="Boissons non-alcoolisées" data-default-src="https://images.yazio.com/drinksnonalcoholic.jpg?w=156&amp;h

Par exemple ici, lorsqu'on clique sur la catégorie 'Boissons alcoolisées', on est redirigé vers le lien https://images.yazio.com/drinksalcoholic.jpg?w=312&amp;h=176&amp;cf . 
On souhaite récupérer ce lien. 

On s'interroge sur le nombre de catégories sur l'interface: 

In [48]:
print("Le nombre de catégories d'aliments est de " + str(len(clean_yazio_page.findAll("a",{"class": "position-relative text-decoration-none"}))))


Le nombre de catégories d'aliments est de 26


On crée une fonction qui nous renvoie les liens pour une certaine catégorie en input.

In [50]:
links = clean_yazio_page.findAll("a",{"class": "position-relative text-decoration-none"})

def get_page(category):
    for i in links : 
        link = i.get("title")
        if link.lower() == category:
            return i.get("href")

Test de la fonction avec la catégorie "Divers"

In [51]:
print(get_page("divers"))

https://www.yazio.com/fr/aliments/divers


On veut maintenant avancer d'un niveau dans le site, et récupérer les liens des sous-catégories (en tout il y a 3 niveaux dans le site et donc 3 types de liens). 

In [54]:
rows = clean_yazio_page.findAll("a",{"class": "position-relative text-decoration-none"})
A = np.zeros((26,2))
a = 0
for i in links : 
    print(i.get("title"),i.get("href"))

Boissons alcoolisées https://www.yazio.com/fr/aliments/boissons-alcoolisees
Boissons non-alcoolisées https://www.yazio.com/fr/aliments/boissons-non-alcoolisees
Divers https://www.yazio.com/fr/aliments/divers
Fast food https://www.yazio.com/fr/aliments/fast-food
Fines herbes et épices https://www.yazio.com/fr/aliments/fines-herbes-epices
Friandises https://www.yazio.com/fr/aliments/friandises
Fromage https://www.yazio.com/fr/aliments/fromage
Fruits https://www.yazio.com/fr/aliments/fruits
Gâteaux et tartes https://www.yazio.com/fr/aliments/gateaux-tartes
Huiles et matières grasses https://www.yazio.com/fr/aliments/huiles-matieres-grasses
Ingrédients pour la pâtisserie https://www.yazio.com/fr/aliments/ingredients-patisserie
Lait et produits laitiers https://www.yazio.com/fr/aliments/lait-produits-laitiers
Légumes https://www.yazio.com/fr/aliments/legumes
Légumineuses https://www.yazio.com/fr/aliments/legumineuses
Muesli, flocons d'avoine et céréales https://www.yazio.com/fr/aliments/mue

On crée un dictionnaire pour réunir les catégories et leurs URL.

In [58]:

dico_categories = dict()
for row in rows : 
    cols = row.get("title")
    if len(cols)>0 : 
        dico_categories[cols]=row.get("href")

On observe le contenu du dictionnaire. 

In [59]:
for link in dico_categories.values():
    print(link)

https://www.yazio.com/fr/aliments/boissons-alcoolisees
https://www.yazio.com/fr/aliments/boissons-non-alcoolisees
https://www.yazio.com/fr/aliments/divers
https://www.yazio.com/fr/aliments/fast-food
https://www.yazio.com/fr/aliments/fines-herbes-epices
https://www.yazio.com/fr/aliments/friandises
https://www.yazio.com/fr/aliments/fromage
https://www.yazio.com/fr/aliments/fruits
https://www.yazio.com/fr/aliments/gateaux-tartes
https://www.yazio.com/fr/aliments/huiles-matieres-grasses
https://www.yazio.com/fr/aliments/ingredients-patisserie
https://www.yazio.com/fr/aliments/lait-produits-laitiers
https://www.yazio.com/fr/aliments/legumes
https://www.yazio.com/fr/aliments/legumineuses
https://www.yazio.com/fr/aliments/muesli-flocons-davoine-cereales
https://www.yazio.com/fr/aliments/noix-graines
https://www.yazio.com/fr/aliments/pate-tartiner-confiture
https://www.yazio.com/fr/aliments/pates-nouilles
https://www.yazio.com/fr/aliments/patisseries-viennoiseries
https://www.yazio.com/fr/alim

On observe les URL des sous-catégories présentes sur une page vers laquelle l'URL des Boissons alcoolisées nous dirige. 

In [60]:

yazio_sous_cat=requests.get("https://www.yazio.com/fr/aliments/boissons-alcoolisees")
clean_yazio_sous_cat = bs4.BeautifulSoup(yazio_sous_cat.content, "lxml")
rows = clean_yazio_sous_cat.findAll("a",{"class": "link"})
for i in rows : 
    print(i.get("href"))
    
#On remarque que certains liens sont en double, on pensera à retraiter plus tard

https://www.yazio.com/fr/aliments/alcools-forts-spiritueux
https://www.yazio.com/fr/aliments/biere
https://www.yazio.com/fr/aliments/cocktails
https://www.yazio.com/fr/aliments/vin
https://www.yazio.com/fr/aliments/lait-de-poule.html
https://www.yazio.com/fr/aliments/champagne-champagne-brut.html
https://www.yazio.com/fr/aliments/champagne-champagne-brut.html
https://www.yazio.com/fr/aliments/lait-de-poule.html
https://www.yazio.com/fr/aliments/lait-de-poule.html
https://www.yazio.com/fr/aliments/champagne-champagne-brut.html


On généralise pour toutes les sous-catégories de toutes les pages et on forme une liste



In [61]:
links = clean_yazio_page.findAll("a",{"class": "position-relative text-decoration-none"})

list_sous_cat = []
for link in dico_categories.values():
    yazio_sous_cat = requests.get(str(link))
    clean_yazio_sous_cat = bs4.BeautifulSoup(yazio_sous_cat.content, "lxml")
    rows = clean_yazio_sous_cat.findAll("a",{"class": "link"})
    for i in rows : 
        list_sous_cat.append(i.get("href"))
        


On observe la liste

In [62]:
for i in range(5):
    print(list_sous_cat[i])

https://www.yazio.com/fr/aliments/alcools-forts-spiritueux
https://www.yazio.com/fr/aliments/biere
https://www.yazio.com/fr/aliments/cocktails
https://www.yazio.com/fr/aliments/vin
https://www.yazio.com/fr/aliments/lait-de-poule.html


On retraite la liste en enlevant les doublons: 

In [63]:
list_sc_retraite =[]

for element in list_sous_cat:
    if element not in list_sc_retraite :
        list_sc_retraite.append(element) 


for i in range(5):
    print(list_sc_retraite[i])

https://www.yazio.com/fr/aliments/alcools-forts-spiritueux
https://www.yazio.com/fr/aliments/biere
https://www.yazio.com/fr/aliments/cocktails
https://www.yazio.com/fr/aliments/vin
https://www.yazio.com/fr/aliments/lait-de-poule.html


On remarque que la liste contient les sous-catégories mais aussi les aliments eux-même, on va donc supprimer les sous-catégories pour n'avoir que les aliments. 
Ces sous-catégories sont les seules à ne pas présenter le suffixe 'html' à la fin de leur lien. 

In [65]:
list_food = []
for element in list_sc_retraite: 
    if element[(len(element)-4):] == 'html':
        list_food.append(element)

for i in range(5):
    print(list_food[i])

https://www.yazio.com/fr/aliments/lait-de-poule.html
https://www.yazio.com/fr/aliments/champagne-champagne-brut.html
https://www.yazio.com/fr/aliments/chocolat-en-poudre.html
https://www.yazio.com/fr/aliments/poudre-de-cacao.html
https://www.yazio.com/fr/aliments/poudre-de-cacao-non-sucree.html


On crée une fonction qui renvoie une liste contenant les 4 nutriments pour un lien donné



In [67]:

def nutri_link(link_aliment):
    tab_indiv = []
    website = requests.get(link_aliment)
    clean_website = bs4.BeautifulSoup(website.content, "lxml")
    linksnb = clean_website.findAll("span",{"class": "serving-calculator-numbers-value"}) #les lignes qui contiennent les valeurs
    #linkstitle = clean_website.findAll("div",{"class": "font-body text-left"}) #les lignes qui contiennent les titres Glucides, Protéines... pour le moment on s'en sert pas
    for i in range(4): #on donne les 4 valeurs kcal, glu, prot, lip
        tab_indiv.append(linksnb[i].get("data-standard-value")) #data-standard-value donne la version pour 100g
    return np.array([tab_indiv])


On essaie et on voit que ça fonctionne



In [68]:
chocopoudre = nutri_link('https://www.yazio.com/fr/aliments/chocolat-en-poudre.html')
print(chocopoudre)


[['400' '90.28' '4.55' '2.27']]


### Création du dataframe

On va maintenant créer un data frame pour concaténer toutes nos données



In [70]:
df = pandas.DataFrame(chocopoudre, index = ['https://www.yazio.com/fr/aliments/chocolat-en-poudre.html'], columns = ['Kcalories', 'Glucides', 'Protéines', 'Lipides'])
df

,Kcalories,Glucides,Protéines,Lipides
https://www.yazio.com/fr/aliments/chocolat-en-poudre.html,400,90.28,4.55,2.27


On trouve un moyen de réécrire le lien avec uniquement le nom de l'aliment

In [23]:
import re
string = re.sub("https://www.yazio.com/fr/aliments/|\.html","",'https://www.yazio.com/fr/aliments/chocolat-en-poudre.html')
print(string)

chocolat-en-poudre


On récapitule tout et on réécrit une ligne de code pour avoir un dataframe meilleur



In [24]:
string = re.sub("https://www.yazio.com/fr/aliments/|\.html","",'https://www.yazio.com/fr/aliments/chocolat-en-poudre.html')
chocopoudre = nutri_link('https://www.yazio.com/fr/aliments/chocolat-en-poudre.html')
df = pandas.DataFrame(chocopoudre, index = [string], columns = ['Kcalories', 'Glucides', 'Protéines', 'Lipides'])
df


,Kcalories,Glucides,Protéines,Lipides
chocolat-en-poudre,400,90.28,4.55,2.27


#### On crée notre tableau final avec tous les aliments

Attention c'est très long

In [73]:
tab_final = pandas.DataFrame(columns = ['Kcalories', 'Glucides', 'Protéines', 'Lipides']) #On crée un dataframe vide avec uniquement les colonnes
for link in list_food : 
    if str(requests.get(link)) != '<Response [404]>':
        string = re.sub("https://www.yazio.com/fr/aliments/|\.html","",link) #on crée un bon titre
        df_nutri= pandas.DataFrame(nutri_link(link),columns = ['Kcalories', 'Glucides', 'Protéines', 'Lipides'], index=[string])
        tab_final = tab_final.append(df_nutri) #on ajoute au dataframe



KeyboardInterrupt: 

In [80]:
tab_final.head(5)

,Kcalories,Glucides,Protéines,Lipides
lait-de-poule,88,8.05,4.55,4.19
champagne-champagne-brut,70,15,0,0


On crée le fichier csv 'tab_final.csv' contenant ce dataframe final. 

In [81]:
import csv
tab_final.to_csv('tab_final.csv')

Nous terminons ainsi la création de notre database contenant les macro nutriments de plus de 1000 aliments. 